# 🛠️ GitHub 모델을 활용한 고급 도구 사용 (.NET)

## 📋 학습 목표

이 노트북은 Microsoft Agent Framework를 .NET에서 GitHub 모델과 통합하여 엔터프라이즈급 도구 통합 패턴을 보여줍니다. 여러 전문화된 도구를 활용하여 강력한 에이전트를 구축하는 방법을 배우며, C#의 강력한 타입 시스템과 .NET의 엔터프라이즈 기능을 활용합니다.

**마스터할 고급 도구 기능:**
- 🔧 **다중 도구 아키텍처**: 여러 전문화된 기능을 가진 에이전트 구축
- 🎯 **타입 안전 도구 실행**: C#의 컴파일 타임 검증 활용
- 📊 **엔터프라이즈 도구 패턴**: 프로덕션 준비된 도구 설계 및 오류 처리
- 🔗 **도구 구성**: 복잡한 비즈니스 워크플로우를 위한 도구 결합

## 🎯 .NET 도구 아키텍처의 장점

### 엔터프라이즈 도구 기능
- **컴파일 타임 검증**: 강력한 타입 시스템으로 도구 매개변수의 정확성 보장
- **의존성 주입**: IoC 컨테이너 통합을 통한 도구 관리
- **Async/Await 패턴**: 적절한 리소스 관리를 통한 비차단 도구 실행
- **구조화된 로깅**: 도구 실행 모니터링을 위한 내장 로깅 통합

### 프로덕션 준비 패턴
- **예외 처리**: 타입화된 예외를 활용한 포괄적인 오류 관리
- **리소스 관리**: 적절한 폐기 패턴 및 메모리 관리
- **성능 모니터링**: 내장된 메트릭 및 성능 카운터
- **구성 관리**: 검증된 타입 안전 구성

## 🔧 기술 아키텍처

### 핵심 .NET 도구 구성 요소
- **Microsoft.Extensions.AI**: 통합된 도구 추상화 계층
- **Microsoft.Agents.AI**: 엔터프라이즈급 도구 오케스트레이션
- **GitHub 모델 통합**: 연결 풀링을 지원하는 고성능 API 클라이언트

### 도구 실행 파이프라인
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ 도구 카테고리 및 패턴

### 1. **데이터 처리 도구**
- **입력 검증**: 데이터 주석을 활용한 강력한 타입 시스템
- **변환 작업**: 타입 안전 데이터 변환 및 포맷팅
- **비즈니스 로직**: 도메인별 계산 및 분석 도구
- **출력 포맷팅**: 구조화된 응답 생성

### 2. **통합 도구**
- **API 커넥터**: HttpClient를 활용한 RESTful 서비스 통합
- **데이터베이스 도구**: Entity Framework를 활용한 데이터 액세스
- **파일 작업**: 검증된 안전한 파일 시스템 작업
- **외부 서비스**: 서드파티 서비스 통합 패턴

### 3. **유틸리티 도구**
- **텍스트 처리**: 문자열 조작 및 포맷팅 유틸리티
- **날짜/시간 작업**: 문화권을 고려한 날짜/시간 계산
- **수학적 도구**: 정밀 계산 및 통계 작업
- **검증 도구**: 비즈니스 규칙 검증 및 데이터 확인

## ⚙️ 사전 준비 및 설정

**개발 환경:**
- .NET 9.0 SDK 이상
- Visual Studio 2022 또는 C# 확장이 포함된 VS Code
- GitHub 모델 API 액세스

**필요한 NuGet 패키지:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**환경 구성 (.env 파일):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

.NET에서 강력하고 타입 안전한 도구 기능을 갖춘 엔터프라이즈급 에이전트를 구축할 준비가 되셨나요? 전문적인 솔루션을 설계해 봅시다! 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
